In [1]:
#!pip install tensorflow

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time

In [3]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10) # No softmax needed due to from_logits=True in loss
    ])

In [4]:
def get_datasets():

    def preprocess(data):
        # Normalize and ensure correct type
        image = tf.cast(data['image'], tf.float32) / 255.
        return image, data['label']

    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )

    return ds_train, ds_test

In [5]:
learning_rate = 1e-3
batch_size = 128
num_epochs = 10

train_ds, test_ds = get_datasets()
train_ds = train_ds.cache().shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)


model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.W20KL0_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.W20KL0_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
print("Starting TensorFlow/Keras training...")
start_time = time.time()

# Training using the high-level fit() function
history = model.fit(
    train_ds,
    epochs=num_epochs,
    validation_data=test_ds,
    verbose=2 # Shows one line per epoch
)

end_time = time.time()

print("-" * 30)
print(f"TensorFlow/Keras Training Time: {end_time - start_time:.4f} seconds")
print("-" * 30)

Starting TensorFlow/Keras training...
Epoch 1/10
469/469 - 12s - 26ms/step - accuracy: 0.9510 - loss: 0.3251 - val_accuracy: 0.9842 - val_loss: 0.0490
Epoch 2/10
469/469 - 2s - 4ms/step - accuracy: 0.9862 - loss: 0.0450 - val_accuracy: 0.9878 - val_loss: 0.0359
Epoch 3/10
469/469 - 3s - 6ms/step - accuracy: 0.9905 - loss: 0.0295 - val_accuracy: 0.9898 - val_loss: 0.0327
Epoch 4/10
469/469 - 2s - 4ms/step - accuracy: 0.9930 - loss: 0.0212 - val_accuracy: 0.9908 - val_loss: 0.0324
Epoch 5/10
469/469 - 2s - 4ms/step - accuracy: 0.9936 - loss: 0.0192 - val_accuracy: 0.9897 - val_loss: 0.0327
Epoch 6/10
469/469 - 3s - 5ms/step - accuracy: 0.9953 - loss: 0.0143 - val_accuracy: 0.9911 - val_loss: 0.0361
Epoch 7/10
469/469 - 2s - 4ms/step - accuracy: 0.9960 - loss: 0.0116 - val_accuracy: 0.9897 - val_loss: 0.0378
Epoch 8/10
469/469 - 2s - 3ms/step - accuracy: 0.9959 - loss: 0.0119 - val_accuracy: 0.9889 - val_loss: 0.0429
Epoch 9/10
469/469 - 2s - 3ms/step - accuracy: 0.9962 - loss: 0.0121 - v

In [7]:
!nvidia-smi

Wed Jul 30 17:47:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             42W /   70W |     382MiB /  15360MiB |     35%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [9]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True